In [300]:
import pandas as pd
dataset=pd.read_csv('../tratamento/dataset_tratado.csv')

dataset_ibge=pd.read_csv('../tratamento/dataset_ibge_tratado.csv')

In [301]:
#Removendo caracteres indesejados no Nome 
dataset['Nome'] = dataset['Nome'].replace({',': ''}, regex=True)
dataset['Nome'] = dataset['Nome'].replace({'-': ''}, regex=True)
dataset = dataset.replace({'  ': ' '}, regex=True)
dataset['Nome'] = dataset['Nome'].replace({"'":' '}, regex=True)



In [302]:
#Valida os CPFS

from validate_docbr import CPF
#Valida os CPFS 
cpf = CPF()
def validaCPF(CPF):
    
    if(cpf.validate(CPF)):
        CPFValido ="Sim"
    else:
        CPFValido ="Não"

        
    return CPFValido

dataset['Validade CPF']=dataset['cpf'].apply(validaCPF)


In [303]:
#Seleciona apenas os CPFs validos
dataset=dataset.loc[dataset['Validade CPF']=="Sim"]
dataset.drop(columns=['Validade CPF'],inplace=True)


In [304]:
#Utiliza Regex para pegar o utimo conjunto de caracteres do nome depois do espaço
import re

def retornaSobrenome(nome): 

  Sobrenome = re.search("[^\/]\w+$", nome).group(0)
  #print(Sobrenome)
  return Sobrenome

dataset['sobrenome']=dataset['Nome'].apply(retornaSobrenome)

In [305]:
#fazendo um join com a base do IBGE para adionar os campos Cod_IBGE,sigla , Estado 
dataset=pd.merge(dataset,dataset_ibge,how='left',on='Cidade')

In [306]:
#Renomeando as colunas 
dataset.rename(columns={'cpf':'CPF','sobrenome':'Sobrenome','cod_ibge':'Cod_IBGE',
'sigla':'Sigla','Nome Sigla':'Estado'},inplace=True)


In [307]:
Dim_localidade=dataset.loc[:,['Cod_IBGE','Sigla','Estado','Cidade']]
dataset.drop(columns=['Sigla','Estado','Cidade'],inplace=True)




,Cod_IBGE,Sigla,Estado,Cidade
0,12,AC,Acre,Rio Branco
1,27,AL,Alagoas,Maceió
2,16,AP,Amapá,Macapá
3,13,AM,Amazonas,Manaus
4,29,BA,Bahia,Salvador
...,...,...,...,...
9995,23,CE,Ceará,Fortaleza
9996,53,DF,Distrito Federal,Brasília
9997,32,ES,Espírito Santo,Vitória
9998,52,GO,Goiás,Goiânia


In [308]:
Fact_indicados=dataset.loc[dataset['Tipo Cargo']=="Indicado"]

In [309]:
df01=dataset.loc[dataset['Tipo Cargo']=="Eleito"]
df02=dataset.loc[dataset['Tipo Cargo']=="Concursado"]

Dim_indicadores=pd.concat([df01,df02])


In [310]:
df=Fact_indicados.merge(Dim_indicadores,how='inner',on=['Sobrenome','Cidade'])
df['Nepotismo']="Sim"

df3=df.loc[:,['Nome_x','Cargo_x','Tipo Cargo_x','CPF_x','Sobrenome','Nepotismo','Cod_IBGE_x']]
df3.rename(columns={'CPF_x':'CPF','Tipo Cargo_x':'Tipo Cargo','Cod_IBGE_x':'Cod_IBGE',
'Cargo_x':'Cargo','Nome_x':'Nome'},inplace=True)
df4=df.loc[:,['Nome_y','Cargo_y','Tipo Cargo_y','CPF_y','Sobrenome','Nepotismo','Cod_IBGE_y']]
df4.rename(columns={'CPF_y':'CPF','Tipo Cargo_y':'Tipo Cargo','Cod_IBGE_y':'Cod_IBGE',
'Cargo_y':'Cargo','Nome_y':'Nome'},inplace=True)





In [311]:
Fact_indicados=Fact_indicados.loc[~Fact_indicados['CPF'].isin(df3['CPF'])]
Fact_indicados['Nepotismo']="Não"
Dim_indicadores=Dim_indicadores.loc[~Dim_indicadores['CPF'].isin(df4['CPF'])]
Dim_indicadores['Nepotismo']="Não"


In [312]:
Fact_indicados=pd.concat([Fact_indicados,df3])
Dim_indicadores=pd.concat([Dim_indicadores,df4])



In [313]:
Fact_indicados.drop_duplicates(inplace=True)
Dim_indicadores.drop_duplicates(inplace=True)

In [314]:
Dim_indicadores.to_csv(r'../processamento/Dim_indicadores.csv', index=False)
Fact_indicados.to_csv(r'../processamento/Fact_indicados.csv', index=False)
Dim_localidade.to_csv(r'../processamento/Dim_localidade.csv', index=False)